# FIAP - Fase 3: VectorStore Generation

This notebook generates the ChromaDB vector store for RAG using Google Colab (A100 40GB GPU).

## Output

After completion, download `chroma_db` folder from Drive to `data/` folder.


In [ ]:
# mount google drive to store populated vectorstore
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install dependencies
!pip install -q chromadb==0.5.23 sentence-transformers==2.7.0 gdown==5.2.0

In [ ]:
# download dataset
!mkdir -p data/pubmedqa/data
!gdown --fuzzy 'https://drive.google.com/file/d/15v1x6aQDlZymaHGP7cZJZZYFfeJt2NdS/view' -O data/pubmedqa/data/ori_pqaa.json

In [ ]:
# configuration
DATA_PATH = "data/pubmedqa/data/ori_pqaa.json"
OUTPUT_DIR = "/content/drive/MyDrive/fiap-3-model"
CHROMA_DIR = f"{OUTPUT_DIR}/chroma_db"
EMBEDDINGS_BATCH = 512
CHROMA_BATCH = 40000

In [ ]:
# load embedding model
from sentence_transformers import SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

In [ ]:
# load dataset
import json
import time

with open(DATA_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

items = list(data.items())

In [ ]:
# prepare documents
documents = []
metadatas = []
ids = []

for pmid, entry in items:
    question = entry.get("QUESTION", "")
    contexts = entry.get("CONTEXTS", [])
    labels = entry.get("LABELS", [])

    doc_text = f"Question: {question}\n\n"
    for context, label in zip(contexts, labels):
        doc_text += f"{label}: {context}\n\n"

    documents.append(doc_text)
    ids.append(pmid)
    metadatas.append({
        "pmid": pmid,
        "question": question,
        "year": str(entry.get("YEAR", "N/A")),
        "meshes": ",".join(entry.get("MESHES", [])),
        "labels": ",".join(labels),
        "contexts": "|||".join(contexts)
    })

In [ ]:
# generate embeddings
start_time = time.time()
total = len(documents)

all_embeddings = []
for i in range(0, total, EMBEDDINGS_BATCH):
    batch = documents[i:i + EMBEDDINGS_BATCH]
    batch_embeddings = embedding_model.encode(batch, show_progress_bar=False)
    all_embeddings.extend(batch_embeddings.tolist())

    processed = min(i + EMBEDDINGS_BATCH, total)
    elapsed = time.time() - start_time
    rate = processed / elapsed
    remaining = (total - processed) / rate if rate > 0 else 0
    print(f"\r{processed:,}/{total:,} ({rate:.0f} docs/s, ~{remaining/60:.1f}min left)", end="")

In [ ]:
# create ChromaDB
import os
import chromadb

os.makedirs(CHROMA_DIR, exist_ok=True)

client = chromadb.PersistentClient(path=CHROMA_DIR)

try:
    client.delete_collection("pubmedqa")
except:
    ...  # no pubmedqa collection

collection = client.create_collection(
    name="pubmedqa",
    metadata={"hnsw:space": "cosine"}
)

total = len(documents)
for i in range(0, total, CHROMA_BATCH):
    end_index = min(i + CHROMA_BATCH, total)
    collection.add(
        ids=ids[i:end_index],
        embeddings=all_embeddings[i:end_index],
        documents=documents[i:end_index],
        metadatas=metadatas[i:end_index]
    )

    print(f"\r{end_index:,}/{total:,} added to ChromaDB", end="")

In [ ]:
# end runtime to prevent wasting credits
from google.colab import runtime
runtime.unassign()